In [1]:
import ee
ee.Initialize()

In [2]:
import geepy
import math
import sys

In [3]:
# Teste 1:

# Todas as classes todo o periodo
# Floresta Período chuvoso

config = geepy.params.configParams('input_classification_v16.json')

amostras = ee.FeatureCollection(config.params['samples']['2016v8'])

watersheds = ee.FeatureCollection(config.params['studyArea']) 

pivots = {i: ee.FeatureCollection(config.params['pivots'] + i) for (i) in config.params['years2process']}

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [4]:
# Modelo digital de elevação
srtm = ee.Image(config.params['srtm'])

# Localities distance
towns = ee.FeatureCollection(config.params['towns'])
rivers = ee.FeatureCollection(config.params['rivers'])

dtown = towns.distance(config.params['radist'])
driver = rivers.distance(config.params['radist'])

slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()
hillshade = ee.Terrain.hillshade(srtm)

ntl30m = {}
for i in config.params['years2process']:
    viirs = ee.Image(config.params['VIIRS'][i]).select('avg_rad').divide(100)
    ntl30m[i] = viirs.resample('bilinear').reproject(
        crs = viirs.projection().crs(),
        scale = 30
    )

In [5]:
landsat = {}

for year in config.params['years2process']:  
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']


    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l8'

    ### add cloud free
    #cloudFreeImg = filtered.map(geepy.image.cloudFree)
    
    fEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ntl30m[year], 'ntl')
    
    fEdgeRemoved = geepy.image.calcNDBI(fEdgeRemoved)

    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, srtm, 'srtm')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, slope, 'slope')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, aspect, 'aspect')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, hillshade, 'hillshade')
    
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dtown, 'dtown')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, driver, 'driver')
    
    fEdgeRemoved = geepy.image.calcNDVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcEVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcSAVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDWI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcMNDWI(fEdgeRemoved)

    fEdgeRemoved = geepy.image.calcFractions(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI2(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI3(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcFCI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcGVNPV(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNPVSOIL(fEdgeRemoved)
    
    fEdgeRemoved = geepy.image.tassCapTransformation(fEdgeRemoved, satellite)

    ndvithermal = fEdgeRemoved.select('ndvi').divide(fEdgeRemoved.select('thermal'))
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ndvithermal, 'ndvithermal')
    
    landsat[year] = fEdgeRemoved.clip(watersheds) 
    
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

Processing Landsat data: 2016

In [6]:
amostragem = geepy.feature.vec2rast(amostras, 'CLASS')

n = 4000
classBand = 'CLASS'
cv = [1,2,3,4,5,6,7]
cp = [n for i in range(len(cv))]

cp[3] = 0    # Agricultura irrigada
#cp[4] = 400 # Pastagem
cp[6] = 200  # Área urbana

samples = geepy.image.randomSamples(amostras, amostragem, n, 369, classBand, cv, cp)
# 1 Formações florestais --- green
# 2 Formações savânicas ---- lightgreen
# 3 Agricultura Sequeiro --- pink
# 4 Agricultura Irrigada --- magenta
# 5 Pastagem --------------- yellow
# 6 Corpos d'água ---------- blue
# 7 Área urbana ------------ red

In [7]:
for year in config.params['years2process']:
    pivots[year] = geepy.feature.vec2rast(pivots[year], 'CLASS').reproject(
        crs = landsat[year].select('nir').projection().crs(),
        scale = 30
    )
    sys.stdout.write("\rProcessing Central Pivots data: %s" % year)
    sys.stdout.flush()

Processing Central Pivots data: 2016

In [8]:
training = samples.filter(ee.Filter.gt('randCol', 0.5))
#validation = samples.filter(ee.Filter.lt('randCol', 0.5))

trained = geepy.image.trainingSamples(landsat['2016'], training)

In [9]:
# Randon Forest
classification = {year: geepy.image.randomForest(landsat[year], trained, bands, ntrees=20) for (year) in config.params['years2process']}

In [10]:
classRemapped = {year: classification[year][0].remap([1,2,3,4,5,6,7],[1,2,3,3,5,6,7]).rename('classification'+year) for (year) in config.params['years2process']}

In [11]:
finalClassification = {year: classRemapped[year].where(pivots[year].select('CLASS'), 4) for year in config.params['years2process']}

In [14]:
mapa = geepy.maps.geeMap(watersheds, zoom=10)

# Training

# 1 Flormações florestais -------------------------------- green
# 2 Formações savânicas ---------------------------------- lightgreen
# 3 Agricultura de Sequeiro ------------------------------ pink
# 4 Agricultura Irrigada --------------------------------- magenta
# 5 Pastagem --------------------------------------------- yellow
# 6 Corpos d'água ---------------------------------------- blue
# 7 Área urbana/Construções rurais ----------------------- red

viz_params = {'min':1,'max':7,'palette':'#004000, #34f200, #ffcaff, #ff42f9, #f4f383, #0000ff, #ff0000','format':'png'}
l5_viz_params = {'min':0.15,'max':0.4,'bands':'swir1,nir,red','format':'png'}
mask_params = {'min':0,'max':1,'palette':'#004000, #34f200'}

mapa.addLayer(landsat['1990'], viz_params=l5_viz_params, name='landsat')

#for i in config.params['years2process']:
mapa.addLayer(finalClassification['1990'], viz_params=viz_params, name='2015')

mapa.addControls()

In [15]:
coords = [[[-46.632916356914535, -15.447083782725066],\
   [-43.13041651144095, -15.447083782725066],\
   [-43.13041651144095, -10.181249376641395],\
   [-46.632916356914535, -10.181249376641395],\
   [-46.632916356914535, -15.447083782725066]]]

In [16]:
# To drive
tasks = {year: geepy.image.send2drive(finalClassification[year], coords, 'classification'+year+'v16', 'classification_v16', 30) for year in config.params['years2process']}

In [17]:
for i in tasks.keys():
   [tasks[i].start()]

In [18]:
accuracy = {year: geepy.image.accuracyAssessment(classification[year][1]) for year in config.params['years2process']}

In [19]:
accTasks = {year: geepy.feature.send2drive(accuracy[year], 'accuracyAssessment' + year, 'accuracyAssessment_v16', 'accuracyAssessment'+year+'_v16', 'GeoJSON') for year in config.params['years2process']}

In [20]:
for i in accTasks.keys():
    [accTasks[i].start()]